# Spark Assignment 1: Analyze precipitation data 

This notebook shows you the basic steps when analyzing precipitation data in a notebook. You will learn how to load a data file to a notebook, access this data file in an Object Storage instance, and then load the content of the data file into a `pandas` DataFrame. Furthermore, you will learn how to explore and analyze the data in the DataFrame and how to vizualize your results by using the `matplotlib` library. 

## Table of contents
1. [Get data](#data_set)

2. [Load data](#load_data)

3. [Access data](#access_data)

4. [Explore data](#explore_data)

5. [Plot the results](#plot_data)

6. [Download the notebook](#download)

7. [Summary](#summary)
-----
<font color='red'>IMPORTANT: </font>This notebook contains credential information, which should be private and confidential. Plase only use it for assignment. Do not share with others and out of course life cycle. 

<a id="data_set"></a> 
## 1. Get data
In this notebook, you will explore and analyze historical annual precipitation data.

The raw precipitation data set is from [UNdata](http://data.un.org/), an Internet search engine for statistical databases provided by the United Nations Statistics Division. You will use a curated version of this data set in this sample. The precipitation measurements are in million cubic meters.

To get the precipitation data set:
1. Click [annual precipitation data set](https://cdsax.cloudant.com/public-samples/test/precipitation.csv) to download the data in CSV format.   
    Note: If you use Safari, right-click the link and select **Download Linked File**. The CSV file is downloaded to your `Download` folder.              
1. Save the CSV file to your computer.



<a id="load_data"></a>
## 2. Load data
You can add the CSV file that you downloaded to this notebook by dragging the file to the **Data Source** pane in the notebook **Palette** in the notebook. 
The data file is listed on the **Data Source** pane and is saved in the Object Storage instance associated with your Analytics for Apache Spark service.
The progress bar below the file in the **Data Source** panel indicates the status of the load process. In the next step, you will learn how to access files in Object Storage.

<a id="access_data"></a> 
## 3. Access data 

**[pandas](http://pandas.pydata.org/)** is a useful Python package for managing structured data.  The two primary data structures of pandas are [`Series`](http://pandas-docs.github.io/pandas-docs-travis/dsintro.html#series) (1-dimensional) and [`DataFrame`](http://pandas-docs.github.io/pandas-docs-travis/dsintro.html#dataframe) (2-dimensional).

In the following steps, you will load the `precipitation.csv` file into a DataFrame in memory.

To run code cells in a notebook, click Run Cell (**&#9658;**) in the notebook toolbar.  While the code in the cell is running, a `[*]` appears next to the cell. After the code has run, the `[*]` is replaced by a number indicating that the code cell is the *N*th cell to run in the notebook.

### Import packages to the notebook

Import the `requests`, `base64`, `StringIO`, `pandas` and `re` packages to use in the notebook. The  pandas package is traditionally imported as `pd`: 

In [ ]:
import requests, StringIO, pandas as pd, json, re

###  Access Object Storage

Because the `precipitation.csv` file is located in Object Storage, you need to define a helper function to access the data file that you loaded. Run the following cell to define the method `get_file_content()`:

In [ ]:
def get_file_content(credentials):
    """For given credentials, this functions returns a StringIO object containing the file content."""
    
    url1 = ''.join([credentials['auth_url'], '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()    
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                if(e2['interface']=='public'and e2['region']==credentials['region']):
                    url2 = ''.join([e2['url'],'/', credentials['container'], '/', credentials['filename']])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO.StringIO(resp2.content)

### Insert data source credentials

To add the credentials to access the `precipitation.csv` file which is stored in Object Storage, click the next code cell and select the `Insert to code` function below the data file in the **Data Source** pane in the notebook palette.

The credentials for accessing the `precipitation.csv` file are added to the cell as a Python dictionary. With these credentials, you can use the helper function to load the data file into a `pandas.DataFrame`. 

**Note**: When you select the `Insert to code` function, a code cell with a dictionary is created for you.  Adjust the credentials in the Python dictionary to correspond with the credentials inserted by the `Insert to code` function and run the dictionary code cell. The access credentials to the Object Storage instance in the dictionary are provided for convenience for later usage. 

In [ ]:
credentials = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_9de0bf59_5fe2_4f11_8d80_ac12b45f3fca',
  'project_id':'3fd339f1bd7b4ff4a1eb8da7659e39ba',
  'region':'dallas',
  'user_id':'46476b565b864525a6a365d41fe7a280',
  'domain_id':'29cedac5a5ca492c83b9c6f324f7229d',
  'domain_name':'913169',
  'username':'Admin_28154f3d777dc89d3deb5a8285de9513c30049b3',
  'password':"""HQ.7BPLU5W1=5Yob""",
  'filename':'precipitation.csv',
  'container':'notebooks',
  'tenantId':''
}

### Load data into `pandas DataFrame`

Run the next cell to load the data into a pandas DataFrame: 

In [ ]:
content_string = get_file_content(credentials)
precipitation_df = pd.read_csv(content_string)

<a id="explore_data"></a> 
## 4. Explore data

Now that your data is in memory, you can begin to explore and manipulate it.

Show the first five and the last five rows of the data by using the `head()` and `tail()` methods.  Run each code cell:

In [ ]:
precipitation_df.head()

In [ ]:
precipitation_df.tail()

Each row in the table lists:

* The country or region where the measurements were taken
* The annual precipitation for 1990, and for the years 1995 to 2009

By using the `DataFrame` API, you can list all countries or areas for which precipitation measurements exist. Run the following cell to list these countries:

In [ ]:
precipitation_df['Country or Area'].values

Having the names of the countries or areas as a column comes in handy, but it would be more useful to have the names as an index to the rows. To manipulate the `DataFrame` in this way, run the next cell:

In [ ]:
precipitation_df = precipitation_df.set_index(precipitation_df["Country or Area"])
precipitation_df.drop(['Country or Area'], axis=1, inplace=True)
precipitation_df.head()

<a id="plot_data"></a> 
## 5. Plot the results

When you work in notebooks, you can decide how to present your anlysis results and derived information. So far, you have used normal print functions, which are informative. However, you can also show your results in a visual way by using the popular [`matplotlib`](http://matplotlib.org/) package to create plots.

First, render your results as inline charts:

In [ ]:
%matplotlib inline 

Now use `pandas` wrappers around the `matplotlib` routines. Because you changed the index of the table rows, you can access the precipitation measurements of a country or area by name. For example, run the following cell to plot the annual precipitation of Germany:

In [ ]:
germany = precipitation_df.ix['Germany']
ax = germany.plot(kind='bar', figsize=(8,5), title="Precipitation of %s between 1990 to 2009" % germany.name)
ax.set_ylabel("Precipitation (million cubic meters)")
ax.set_xlabel("Years");

Another possibility is to import the `matplotlib` library and use it directly for plotting the precipitation values in France:

In [ ]:
import matplotlib.pyplot as plt, numpy as np

france = precipitation_df.ix['France']
years = france.index
index = np.arange(len(years))  
plt.figure(figsize=(8,5))
bar_width = 0.5
plt.bar(index, france, bar_width, color='b')
plt.xlabel("Year")
plt.ylabel("Precipitation (million cubic meters)")
plt.title("Precipitation of %s between 1990 to 2009" % france.name, y=1.08)
plt.xticks(index + bar_width, years, rotation=90)
plt.show()

### Plot the countries or areas with the highest total precipitation

To get a list of the countries with the highest total precipitation, you have to compute the sum of all annual precipitation. You can do this by using the `pandas` `sum()` method. This method adds an  additional column called SUM, which contains the sums of the annual precipitation values to the `precipitation_df` DataFrame.

In [ ]:
precipitation_df["SUM"] = precipitation_df.sum(axis=1)

Next, sort the DataFrame by total precipitation and print the top five countries or areas with the highest total precipitation:

In [ ]:
precipitation_sorted_df = precipitation_df.sort_values(by="SUM",ascending=False)
top5_sums = pd.Series(precipitation_sorted_df["SUM"].head(5))
top5_sums

Now, plot the top five countries next to each another in a line graph for a better comparison of the recordings:

In [ ]:
top5_bars = precipitation_sorted_df[years][0:5].transpose()
ax = top5_bars.plot(figsize=(10,8), marker='o', linestyle='-', title="Top 5 Countries with highest Precipitation")
ax.set_xlabel("Years")
ax.set_ylabel("Precipitation (million cubic meters)");

Finally, compare the annual precipitation for the top five countries. China has the highest annual precipitation, followed by Colombia. The line graph also shows that some values are missing, which makes comparing measurements difficult. 

To show the total precipitation for the top five countries in relation to the total precipitation for all countries over the entire time period, use a pie chart. Run the next code cell to create a pie chart by using the `matplotlib` library:

In [ ]:
precipitation_sums = top5_sums
other_sums = precipitation_sorted_df["SUM"][5:].sum()
precipitation_sums["Other"] = other_sums

plt.axis('equal')
plt.title("Annual precipitation percentage",y=1.08)
plt.pie(
    precipitation_sums,
    labels=precipitation_sums.index,
    colors=['blue', 'green', 'red', 'turquoise', 'magenta','yellow'],
    autopct="%1.2f%%",
    radius=1.25);

The pie chart shows you that nearly a quarter of the precipitation fell in China, and that more than  half of the precipitation fell in the five countries with the highest annual precipitation.

###  Plot the countries with a negative annual precipitation trend

Going through each row in the DataFrame and looking at the figures is not a viable solution to determine trends. Plotting a bar chart for each country or area is possible, but inconvenient. 

One way to determine trends in annual precipitation is to fit a line to the data points. Run the next code cell to create a bar chart showing the annual precipitation trend for Chile:

In [ ]:
chile = precipitation_sorted_df.ix['Chile'][:-1]
years = chile.index
chile.plot(kind='bar', figsize=(8,5), title="Precipitation of %s between 1990 to 2009" % chile.name)
plt.xlabel("Year")
plt.ylabel("Precipitation (million cubic meters)");

A line of best fit is a straight line that is the best approximation of the given set of data.
The `polyfit()` function fits a line through data points. Because the DataFrame you are using is two-dimentional, it returns two values, the slope and the intercept. To determine whether the trend is positive or negative, you need only the slope. And naturally, you have to exclude any data points that correpond to the value 0.0. 

Run the next cells to determine and line plot the trend for Chile:

In [ ]:
index = chile[chile>0.0].index
fit = np.polyfit(index.astype('int'), chile[index].values,1)
print "slope: " + str(fit[0])

In [ ]:
plt.plot(index, chile[index],'.')
plt.plot(index, fit[0]*index.astype('int') + fit[1], '-', color='red')
plt.title("Precipitation Trend for Chile")
plt.xlabel("Year")
plt.ylabel("Precipitation (million cubic meters)")
plt.show()

For Chile, the line plot shows a positive trend in annual precipitation, despite the fact that during the last couple of years the annual precipitation decreased. 

The next cells show you how to determine the precipitation trends for 91 countries.
Run the next code cell to define the `compute_slope()` method to compute the slope of a fitted line  between given points:

In [ ]:
def compute_slope(row):
    """ This method computes the slope of a fittet line for given data points. 
    For arguments containing less than two valid data points it returns a NaN."""
    slope = np.nan
    years = row[row>0.0].index
    if(len(years)>=2):
        fit = np.polyfit(years.astype('int'), row[years].values,1)
        slope = fit[0]
    return slope

After defining the `compute_slope()` method, use the `pandas` `apply()` method to compute the slope for each row in the DataFrame and add the computed slope value to a new column called TREND:

In [ ]:
precipitation_df["TREND"] = precipitation_sorted_df[years].apply(compute_slope, axis=1)
precipitation_df.head()

If the value is positive, the annual precipitation trend for the country is positive. If the value is negative, the annual precipitation trend is negative. Values near zero indicate a stable condition. 

To determine the countries with a negative precipitation trend, find the rows in the DataFrame with a negative value in the TREND column:

In [ ]:
negative_trends = precipitation_df[precipitation_df["TREND"]<0.0]
negative_trends.index

<a id="summary"></a>
## Summary

In this sample, you learned the basic tools for working with notebooks. You learned how to use the preinstalled `pandas` library to explore and analyze precipitation data to gain insights about the distribution of precipitation in 91 countries and detect trends. You learned to use the `matplotlib` libraries to visualize your results in different ways, for example as bar charts, pie charts, and line graphs. 

### Resources

[Learn Python the Hard Way](http://learnpythonthehardway.org/book/)<br/> 
[Python Programming Language](https://www.python.org/)<br/> 
[IPython Notebook](http://nbviewer.ipython.org/github/ipython/ipython/blob/2.x/examples/Notebook/Index.ipynb)<br/> 
[Jupyter Notebook](http://jupyter.readthedocs.org/en/latest/)<br/> 
[A gallery of interesting Notebooks](https://github.com/ipython/ipython/wiki/A-gallery-of-interesting-IPython-Notebooks)<br/> 

Check out these interesting tutorials on YouTube:

[PyData 2015 - Introduction to Pandas](https://www.youtube.com/watch?v=TSsSWuhBpmY)

[Matplotlib Python Tutorial](https://www.youtube.com/watch?v=wAwQ-noyB98)